# How to use newdust scattering models

There are a variety of physical assumptions and numerical methods that can be applied to model the propagation of light through a solid material. The `newdust` library provides allows the user to employ these methodologies in an interchnageable way. The `newdust.scatteringmodel` module provides the following extinction calculators:

+ `newdust.scatteringmodel.Mie()` employs the Bohren & Huffman (1983) Mie-scattering algorithm, which has been sped up through the used of vectorized computations. This can be  demanding on temporary (RAM) memory storage, depending on the wavelength resolution or number of grain radii used. Some care is needed for calculating extinction in high resolution.
+ `newdust.scatteringmodel.RGscattering()` employs the Rayleigh-Gans approximation, which is relevant for dust grains that are much bigger than the wavelength of light and relatively transparent to the incoming light waves (i.e., $|m-1| << 1$, where $m$ is the complex index of refraction)
+ `newdust.scatteringmodel.PAH` reads and interpolates over the tables for the extinction properties of poly-cyclic aromatic hydrocarbons (PAHs) from Li & Draine (2001)

This notebook demonstrates how to set up a dust extinction model and run an extinction calculation. To compute extinction, the scattering model needs to receive a `newdust.graindist.composition.Composition` object, which provides the optical constants for the compound, and an array of grain radii. If no units are attached the the grain radii (via Astropy units), then the input is assumed to have units of microns.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
import astropy.constants as c

import newdust

%matplotlib inline

## Rayleigh-Gans model

This example shows how to compute Rayleigh-Gans scattering with the Drude approximation.

In [ ]:
# Energy grid that we will use to compute the scattering model
ENERGY = np.logspace(-1,1,30) * u.keV

# We'll compute the properties for a single grain size
AUM    = np.array([1.0]) * u.micron

THETA  = np.logspace(-5., np.log10(np.pi), 1000) * u.rad

In [ ]:
# First initialize a Composition object with the Drude approximation
CM = newdust.graindist.composition.CmDrude()

# Initialize the Rayleigh-Gan extinction model
rgd = newdust.scatteringmodel.RGscattering()

# Calculate the extinction for the energy grid assumed above
rgd.calculate(ENERGY, AUM, CM)

The results of the calculation, which are the various attenuation efficiencies, are stored within the `ScatteringModel` object. Efficiency is defined as the physical cross-section divided by the geometric cross-section; for spherical grains the efficiency is:
$$ Q = \frac{\sigma}{\pi a^2} $$
where $\sigma$ is the cross-section for the physical interaction (e.g., scattering) and $a$ is the radius of the dust grain. Efficiency values are unitless.

 + `ScatteringModel.qsca` holds the scattering efficiency
 + `ScatteringModel.qabs` holds the absorption efficiency
 + `ScatteringModel.qext` holds the extinction efficiency (qsca + qabs)
 + `ScatteringModel.diff` holds the differential scattering cross section divided by the geometric cross-section
 
The integrated extinction efficiencies -- `qsca`, `qabs`, and `qext` -- are 2-D numpy arrays with dimensions (NE, NA), where NE is the length of the input wavelength or energy grid, and NA is the length of the input grain size grid.

The differential scattering cross-section, `diff`, is a 3-D numpy array with dimensions (NE, NA, NTH) where NTH is the length of the angular sampling grid (`theta` argument in `ScatteringModel.calculate`).

In [ ]:
plt.plot(ENERGY, rgd.qsca)
plt.loglog()

The Rayleigh-Gans plus Drude approximation returns a scattering cross-section that decays with $E^{-2}$.

## Computing a differential scattering cross-section

To compute the differential cross-section over a range of angular values, one must input an array (with or without units) using the `theta` keyword. If no unit is specified via the astropy units package, the values are assumed to have units of radians.

By default, the argument for `theta` is set to 0. You will only need to input a value for `theta` if you wish to evaluate scattering as a function of angle.

In [ ]:
rgd2 = newdust.scatteringmodel.RGscattering()
rgd2.calculate(ENERGY, AUM, CM, theta=THETA)

Below, I print the shape of the stored efficiency arrays to verify their dimensionality.

$Q_{\rm sca}$ is, by definition, integrated over all scattering angles. So it is a function of wavelength/energy and grain radius. When I print the shape, I see that it has dimensions of `(len(ENERGY), len(AUM))`

The differential scattering cross-section is a function of wavelength/energy, grain radius, and angle. It has dimensions of `(len(ENERGY), len(AUM), len(THETA_ASEC))`

In [ ]:
rgd2.qsca.shape

In [ ]:
rgd2.diff.shape

Seeing the shape of the objects helps to understand how to access the differential cross-section information. 

To see how the differential cross-section changes with energy, I plot the differential scattering cross-section of the lowest energy value (indexed by 0) and the highest energy value (indexed by -1, i.e., the last value in the ENERGY array). 

Since there is only one grain radius value, we use 0 along the 2nd dimension in the array.

In [ ]:
xunit = 'arcmin'
plt.plot(THETA.to(xunit), rgd2.diff[0,0,:], 'b-', lw=2, label=ENERGY[0])
plt.plot(THETA.to(xunit), rgd2.diff[-1,0,:], 'k--', lw=2, label=ENERGY[-1])
plt.title("Grain radius: {}".format(AUM[0]))
plt.xlabel(r'$\theta$ ({})'.format(xunit), size=12)
plt.ylabel(r'$dQ_{\rm sca}/d\Omega$ (ster$^{-1}$)', size=12)
plt.loglog()
plt.legend()

Verify that the differential cross-section integrates to the same scattering efficiency value. Generally, this calculation is accurate on the few percent level.

In [ ]:
i = -1 # index for the energy value to check
integrand = np.trapz(rgd2.diff[i,0,:] * 2.0*np.pi*np.sin(THETA.to('rad').value), 
                THETA.to('rad').value)

print(integrand/rgd2.qsca[i,0])

## Mie model

This example shows how to compute a Mie scattering model for silicate dust grains at a single wavelength.

In [ ]:
# Use a visible wavelength for this calculation
wl_V  = 4500. * u.angstrom

# Initiate the silicate composition object
cm_sil = newdust.graindist.composition.CmSilicate()

In [ ]:
mtest = newdust.scatteringmodel.Mie()
mtest.calculate(wl_V, AUM, cm_sil, theta=THETA)

Note the shape of the computed differential scattering cross-section, now that we are using only 1 wavelength value.

In [ ]:
np.shape(mtest.diff)

In [ ]:
xunit='rad'
plt.plot(THETA.to(xunit), mtest.diff[0,0,:])
plt.semilogy()
plt.xlabel(r'$\theta$ ({})'.format(xunit), size=12)
plt.ylabel(r'$dQ_{\rm sca}/d\Omega$ (ster$^{-1}$)', size=12)

Verify that the differential cross-section integrates to the same scattering efficiency value.

In [ ]:
test = np.trapz(mtest.diff * 2.0*np.pi*np.sin(THETA.to('rad').value), THETA.to('rad').value)
print(test/mtest.qsca)

### Warning: The Mie scattering model can take up memory

The Mie scattering model uses multi-dimensional arrays instead of for-loops to complete a calculation. Unfortunately, this computation can easily exceed the amount of RAM available on a typical laptop or desktop computer, which can crash your system. To avoid crashing, the `newdust.scatteringmodel.Mie.calculate` method estimates the amount of memory the computation will take and prints a warning statement instead of performing the calculation.

The memory limitation is assumed to be 8 GB by default. The below example shows what happens when you hit this limit. If you have more RAM available to your system, you can override the default by setting the `memlim` keyword.

In [ ]:
# High energy calculations take up a LOT of memory. 
# Here is a ridiculous example that will cause us to exceed the default memory limit.
EGRID = np.linspace(1000., 5000., 2) * u.keV  
AVALS = np.linspace(0.1, 0.5, 20) * u.micron

mtest2 = newdust.scatteringmodel.Mie()
mtest2.calculate(EGRID, AVALS, cm_sil, theta=THETA)

**WARNING:** Do not run the following code if you have a memory limited machine (8 GB RAM or less).

According to the printed statement above, this calculation is estimated to take up a little over 8 GB of RAM. My machine has 16 GB of RAM, so I could technically run this calculation. The lines below show you to increase the `memlim` keyword value to 8.3 GB, which will allow the computation should proceed.

```
mtest2 = newdust.scatteringmodel.Mie()
mtest2.calculate(EGRID, AVALS, cm_sil, theta=THETA, memlim=8.3)
```

However, this is a silly example because dust scattering isn't a phenomenon witnessed at 5 GeV.

### Mie Scattering computation with slightly higher dimensionality

The following code gives an example of examining the results of a Mie scattering computation where all three dimensions (energy/wavelength, grain radius, and scattering angle) are utilized.

In [ ]:
LAMVALS = np.linspace(1000.,5000.,5) * u.angstrom
AVALS   = np.linspace(0.1, 0.5, 20) * u.micron

mtest3 = newdust.scatteringmodel.Mie()
mtest3.calculate(LAMVALS, AVALS, cm_sil, theta=THETA)

Now I can compare the differential scattering cross-section for two different wavelengths:

In [ ]:
xunit = 'rad'
plt.plot(THETA.to(xunit), mtest3.diff[0,0,:], label=LAMVALS[0])
plt.plot(THETA.to(xunit), mtest3.diff[-1,0,:], label=LAMVALS[-1])
plt.semilogy()

plt.title("Grain radius: {}".format(AVALS[0]), size=12)
plt.xlabel(r'$\theta$ ({})'.format(xunit), size=12)
plt.ylabel(r'$dQ_{\rm sca}/d\Omega$ (ster$^{-1}$)', size=12)
plt.legend()

or two different grain sizes:

In [ ]:
xunit = 'rad'
plt.plot(THETA.to(xunit), mtest3.diff[0,0,:], label=AVALS[0])
plt.plot(THETA.to(xunit), mtest3.diff[0,-1,:], label=AVALS[-1])
plt.semilogy()

plt.title("Wavelength: {}".format(LAMVALS[0]), size=12)
plt.xlabel(r'$\theta$ ({})'.format(xunit), size=12)
plt.ylabel(r'$dQ_{\rm sca}/d\Omega$ (ster$^{-1}$)', size=12)
plt.legend()